In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import datasets
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import recall_score, matthews_corrcoef, f1_score
from sklearn.model_selection import StratifiedShuffleSplit
#%matplotlib inline
sns.set_style('whitegrid')
print(np.__version__)
print(pd.__version__)
import sys
print(sys.version)
import sklearn
from scipy.stats import ttest_ind
import scipy.stats  as stats
#print(sklearn.__version__)
from statsmodels.stats.weightstats import ztest as ztest
from sklearn.metrics import matthews_corrcoef
import propy
from propy import PyPro
from propy.PyPro import GetProDes
from pathlib import Path  
import pandas as pd
import logging


1.20.3
1.3.4
3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]


In [2]:
# this function returns list of all UNIQUE AMPs and their sequences from Angela's dataset
def AMP_list(df1,file_name):
    import pandas as pd
    amp_list=[ ]
    amp_seq=[ ]
    file_name = open(file_name,"a")

    for j in range(df1.shape[0]):
        element1=df1['peptide I'].loc[j]
        element2=df1['peptide II'].loc[j]
        seq1=df1['seq I'].loc[j]
        seq2=df1['seq II'].loc[j]
        s1=seq1.replace(" ", "")
        s2=str(seq2).replace(" ", "")
        sq1=s1.replace("-", "")
        sq2=s2.replace("-", "")
     #print(type(s1),type(s2))

        if type(element1) is not float and element1 not in amp_list:
        #print("peptide I is:", df1['peptide I'].loc[j],", And seq is:" , df1['seq I'].loc[j])
        #print("peptide II is:", df1['peptide II'].loc[j],", And seq is:" , df1['seq II'].loc[j])
            amp_list.append(element1)
            amp_seq.append(sq1)
            file_name.write('\n%s' % '>'+element1)
            if pd.isnull(seq1) is False :
                print(element1,sq1.upper())
                file_name.write('\n%s' % sq1.upper())

        if type(element2) is not float and element2 not in amp_list:
            amp_list.append(element2)
            amp_seq.append(sq2)
            file_name.write('\n%s' % '>'+element2)
            if pd.isnull(seq2) is False :
                print(element2,sq2.upper())
                file_name.write('\n%s' % sq2.upper())

            
    file_name.close()
        
    return 

In [3]:
# this function returns a dictionary

def AMP_pairs_dict(df1):
    amp_list=[ ]
    fic_list=[ ]
    amp_dicts = {}
    amp_dicts2 = {}
    i=0
    for j in range(df1.shape[0]):
        element1=df1['peptide I'].loc[j]
        element2=df1['peptide II'].loc[j]
        element_fic=df1['FICI'].loc[j]
        #print(element1, element2,element_fic)
 
        if type(element1) is not float and element1 not in amp_list:
            amp_list.append(element1)
            if type(element2) is not float and element2 not in amp_list:
                i=i+1
                amp_list.append(element2)
                #print(element1,element2)
                amp_dicts[element1] = element2
                amp_dicts2[element_fic] = [element1,element2]
                #print(i)
    return  amp_dicts,amp_dicts2

In [6]:
def amp_names(input_file):
    n_data= []
    myfile = open(input_file, 'r')
    for line in myfile:
        if  line.startswith('>'):
            line=line[0:-1]
            newstr = line.replace(">", "")
            #print(newstr)
            n_data.append(newstr)
        
    return n_data

In [7]:
def Euclidean_distance (df,AMP_A,AMP_B):
    D_e=np.linalg.norm(df.loc[AMP_A] - df.loc[AMP_B])
    #D_e=np.linalg.norm(AMP_A - AMP_B)


    return D_e

In [8]:
import numpy as np
# this function returns a dataframe containing AMP pairs, metric distance and corresponding FIC
def metric_amp(df,my_amps):
    name_list2=df.index
    list_fic=[]
    list_d=[]
    for x , y in my_amps[1].items():
        if (y[0] in name_list2):
            amp_distance=Euclidean_distance (df, y[0], y[1])
            #print(x,y[0],y[1],amp_distance)
            list_d.append(amp_distance)
            list_fic.append(x)
        
        
    return list_fic,list_d

In [10]:
def total_charge(proseq):
    
    chargeDict = {"A":0, "C":0, "D":-1, "E":-1, "F":0, "G":0, "H":1, "I":0, "K":1, "L":0, "M":0, "N":0, "P":0, "Q":0, "R":1, "S":0, "T":0, "V":0, "W":0, "Y":0, "X":0}
    netCharge = sum([chargeDict[x] for x in proseq])
    #print(proseq, netCharge)
    return netCharge

In [11]:

def standard_aa_fun(seq):
    
    chargeDict = {"A":0, "C":0, "D":-1, "E":-1, "F":0, "G":0, "H":1, "I":0, "K":1, "L":0, "M":0, "N":0, "P":0, "Q":0, "R":1, "S":0, "T":0, "V":0, "W":0, "Y":0}
    standard_aa=list(chargeDict.keys())
    val=1
    for element in seq:
        if element not in standard_aa:
            val=0
            #print(element,val)
            break
        
    return val   

In [13]:

def All_Descriptors(input_file):
    dfs = [] 
    df=[]
    Name_list=[]
    myfile = open(input_file, 'r')
    i=0
    for line in myfile:
        if line.startswith('>'):
            ll=line
            ll=ll[0:-1]
            newstr = ll.replace(">", "")
        if not line.startswith('>'):
            line=line[0:-1]
            arg=standard_aa_fun(line)
            if arg==1:
                #print("seq")
                print(newstr)
                Name_list.append(newstr)
                i=i+1
                Des=GetProDes(line)
                alldes = Des.GetALL()
                df = pd.DataFrame([alldes])
                dfs.append(df)
                ddf=pd.concat(dfs) 
                #print(Name_list)

    myfile.close()     
    #.reset_index(inplace=True) 
    return ddf,Name_list

In [2]:
def net_charge_calculator(input_file):
    c_data= []
    myfile = open(input_file, 'r')
    
    for line in myfile:
        if line.startswith('>'):
            ll=line
            ll=ll[0:-1]
            newstr = ll.replace(">", "")
        if not line.startswith('>'):
            line=line[0:-1]
            arg=standard_aa_fun(line)
            if arg==1:
                #print("charge")
                #print(newstr)
                line=line[0:-1]
                net_c=total_charge(line)
                c_data.append(net_c)
            
    return pd.DataFrame(c_data,columns=["net_charge"])


In [15]:
def Descriptors_complete(neg,csv_file):
    nc_df=net_charge_calculator(neg)
    all_df,amp_names_list=All_Descriptors(neg)
    df_names=pd.DataFrame(amp_names_list,columns=["AMP_Name"])
    #print(df_names)
    #print(nc_df)
        
    result1 = pd.merge(df_names, nc_df, left_index=True, right_index=True)
    result = pd.merge(result1,all_df.reset_index(drop=True), left_index=True, right_index=True)
    #print(result1)

    filepath = Path(csv_file)  
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    result.to_csv(filepath)  

    return result


In [16]:
def AMP_list2(df1,file_name):
    import pandas as pd
    amp_list=[ ]
    amp_seq=[ ]
    amp_dict1 = {}
    amp_dict2 = {}

    
    file_name = open(file_name,'w')

    for j in range(df1.shape[0]):
        element1=df1['peptide'].loc[j]
        seq1=df1['sequence'].loc[j]
        s1=seq1.replace(" ", "")
        sq1=s1.replace("-", "")
        #print(j,element1)
        if type(element1) is not float and element1 not in amp_list:
            amp_list.append(element1)
            amp_dict1[element1]=df1['MIC'].loc[j]
            amp_dict2[element1]=df1['MIC_0'].loc[j]

            amp_seq.append(sq1)
            file_name.write('>' + element1 )
            #file_name.write('\n%s' % '>' + element1)
            if pd.isnull(seq1) is False :
                #print(element1,sq1.upper())
                file_name.write('\n%s' % sq1.upper() + '\n' )

       
            
    file_name.close()
        
    return amp_dict1,amp_dict2

In [17]:
# this function takes the original dataframe a returns a ready to use dataset
def final_df(df,textfile,csvfile):

    dict1,dict2=AMP_list2(df,textfile)
    #print(dict1)
    df1=Descriptors_complete(textfile, csvfile)
    filepath = Path(csvfile)  
    filepath.parent.mkdir(parents=True, exist_ok=True)      
    #df1=pd.read_csv(csvfile, )
    #df2=df1.drop(columns=['Unnamed: 0'])

    df1['MIC']=list(dict1.values())
    df1['MIC_0']=list(dict2.values())
    
    df1.to_csv(filepath)  

    return 

In [18]:
def normalized_df(df):

    from sklearn.preprocessing import MinMaxScaler
    df0=df.drop(columns=['AMP_Name'])

    scaler = MinMaxScaler()
    df_normalized = scaler.fit_transform(df0)
    scaled_features_df = pd.DataFrame(df_normalized, index=df0.index, columns=df0.columns)

    return scaled_features_df

In [21]:
# this function takes the original dataframe a returns a ready to use dataset
def final_df2(df,textfile,csvfile):

    dict1,dict2=AMP_list0(df,textfile)
    #print(dict1)
    df1=Descriptors_complete(textfile, csvfile)
    filepath = Path(csvfile)  
    filepath.parent.mkdir(parents=True, exist_ok=True)      
    #df1=pd.read_csv(csvfile, )
    #df2=df1.drop(columns=['Unnamed: 0'])

    df1['MIC']=list(dict1.values())
    df1['MIC_0']=list(dict2.values())
    
    df1.to_csv(filepath)  

    return 

In [22]:
def AMP_list0(df1,file_name):
    import pandas as pd
    amp_list=[ ]
    amp_seq=[ ]
    amp_dict1 = {}
    amp_dict2 = {}

    
    file_name = open(file_name,'w')

    for j in range(df1.shape[0]):
        #print(j)
        element1=df1['peptide'].loc[j]
        seq1=df1['sequence'].loc[j]
        s1=seq1.replace(" ", "")
        sq1=s1.replace("-", "")
        #print(j,element1)
        if type(element1) is not float and element1 not in amp_list:
            amp_list.append(element1)
            amp_dict1[element1]=df1['MIC'].loc[j]
            amp_dict2[element1]=df1['MIC_0'].loc[j]

            amp_seq.append(sq1)
            file_name.write('>' + str(element1) )
            #file_name.write('\n%s' % '>' + element1)
            if pd.isnull(seq1) is False :
                #print(element1,sq1.upper())
                file_name.write('\n%s' % sq1.upper() + '\n' )

    file_name.close()
        
    return amp_dict1,amp_dict2

In [23]:
#cross validation
def cross_val_split(X,y,ns):
    i = 0
    from sklearn.model_selection import StratifiedShuffleSplit

    Xtrain_cv = list(range(ns))
    Xtest_cv = list(range(ns))
    Ytrain_cv = list(range(ns))
    Ytest_cv = list(range(ns))
    cv = StratifiedShuffleSplit(n_splits=ns, test_size=0.2, random_state=999)
    for train_index, test_index in cv.split(X, y):
        Xtrain_cv[i] = X[train_index]
        Xtest_cv[i] = X[test_index]
        Ytrain_cv[i] = y[train_index]
        Ytest_cv[i] = y[test_index]
        i+=1
    return Xtrain_cv, Xtest_cv, Ytrain_cv, Ytest_cv

In [24]:
def linear_SVM_grid_search(X,y,ns):
    from sklearn.svm import LinearSVC
    from sklearn.model_selection import train_test_split
    from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
    X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size = 0.2, random_state = 20)
    model = LinearSVC()
    C_range = np.logspace(-1,1,3)
    param_grid = {"C": C_range}
    scoring = ['accuracy']
    kfold = StratifiedShuffleSplit(n_splits=ns, test_size=0.2, random_state=999)

    grid_search = GridSearchCV(estimator = model,
                           param_grid=param_grid,
                           refit='accuracy',
                           n_jobs=-1,
                           cv=kfold,
                           verbose=0)
    grid_result = grid_search.fit(X_train,Y_train)
    CC=grid_result.best_params_['C']
    return CC

In [25]:
def svm_feature_selection(Xtrain_cv,Ytrain_cv,CC,ns,klist):
    from sklearn.svm import LinearSVC
    import numpy as np
    listlocs = list(range(ns))
    results = list(range(ns))
    kfeatures = list(range(ns))
    kweights = list(range(ns))

    bagging_dict = {}
    for i in range(ns): # loop over cross validation sets
        model = LinearSVC()
        params = {"penalty":"l1", "C":CC, "loss":"squared_hinge", "dual":False, "max_iter": 1000000}
        model.set_params(**params)
    
        X_train = Xtrain_cv[i]
        Y_train = Ytrain_cv[i]
# coef_ is a list of weights same as w
        model.fit(X_train, Y_train)
        T = model.coef_[0]
        TT = list(T)
        locs = np.where(abs(T) > 0)
        listlocs[i] = np.asarray(locs)

        if i == 0:
            flistlocs = listlocs[i]
            flistlocs=flistlocs[0]
        else:
            flistlocs = np.intersect1d(flistlocs,listlocs[i])
        

        for f in flistlocs:
            f_feature = klist[f]
            f_weight = T[f]
        
            if i == 0:
                bagging_dict[f_feature] = []
                bagging_dict[f_feature].append(f_weight)
            else:
                bagging_dict[f_feature].append(f_weight)

        kfeatures[i] = klist[listlocs[i]]
        kweights[i] = T[listlocs[i]]
        
    kfeatures=klist[flistlocs]
    featuredict={}
    for k,v in bagging_dict.items():
        if k in kfeatures:
            print(k, sum(v)/len(v))
            featuredict[k]=sum(v)/len(v)
    return featuredict

In [26]:
def svm_feature_plot(featuredict,fig_title,fonts):
    featuredict = sorted(featuredict.items(), key=lambda x:x[1])
    feature_name=[]
    feature_coef=[]

    for items in featuredict:
        #print(items[0], items[1])
        feature_name.append(items[0])
        feature_coef.append(items[1])
    xx=feature_name
    yy=feature_coef
    # Plot of feature importance
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    plt.figure(figsize=(7, 7))
    #sns.set_theme(style="whitegrid")
    ax = sns.barplot(yy, xx)
    plt.xlabel("Feature Importance",  fontsize=fonts)
    
    plt.savefig(fig_title, bbox_inches = 'tight',
        pad_inches = 0)
    plt.show()

In [27]:
def lasso_grid_search(X,y):
    from sklearn.linear_model import Lasso
    from sklearn.model_selection import RepeatedKFold
    from sklearn.model_selection import GridSearchCV
    from numpy import arange
    # define model
    model = Lasso()
    # define model evaluation method
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    # define grid
    grid = dict()
    grid['alpha'] = arange(0, 1, 0.01)
    # define search
    search = GridSearchCV(model, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
    # perform the search
    results = search.fit(X, y)
    AA = results.best_params_['alpha']
    print(AA)
    return AA

In [28]:
def lasso_feature_selection(Xtrain_cv,Ytrain_cv,CC,ns,klist):
    from sklearn.linear_model import Lasso
    import numpy as np
    listlocs = list(range(ns))
    results = list(range(ns))
    kfeatures = list(range(ns))
    kweights = list(range(ns))

    bagging_dict = {}
    model = Lasso(alpha = CC)
    for i in range(ns):
    
        X_train = Xtrain_cv[i]
        Y_train = Ytrain_cv[i]

        model.fit(X_train, Y_train)
        #print(i)
        #print(model.coef_)
        T = model.coef_
        #print(i)
        #print(T)
        TT = list(T)
        locs = np.where(abs(T) > 0)
        listlocs[i] = np.asarray(locs)

        if i == 0:
            flistlocs = listlocs[i]
            flistlocs=flistlocs[0]
        else:
            flistlocs = np.intersect1d(flistlocs,listlocs[i])
        

        for f in flistlocs:
            f_feature = klist[f]
            f_weight = T[f]
        
            if i == 0:
                bagging_dict[f_feature] = []
                bagging_dict[f_feature].append(f_weight)
            else:
                bagging_dict[f_feature].append(f_weight)

        kfeatures[i] = klist[listlocs[i]]
        kweights[i] = T[listlocs[i]]
        
    kfeatures=klist[flistlocs]
    featuredict={}
    for k,v in bagging_dict.items():
        if k in kfeatures:
            print(k, sum(v)/len(v))
            featuredict[k]=sum(v)/len(v)
    return featuredict

In [ ]:
def models(X_train,Y_train):
        #logistic regression
        
        log1=LogisticRegression(random_state=0)
        
        log1.fit(X_train,Y_train)
        
        
        svc_model = SVC()
        svc_model.fit(X_train, Y_train)

        log = log1.score(X_train,Y_train)
        svc = svc_model.score(X_train, Y_train)

        y_pred_log=log1.predict(X_train)
        y_pred_svc=svc_model.predict(X_train)

        rec_log=recall_score(Y_train, y_pred_log, average='macro')
        rec_svc=recall_score(Y_train, y_pred_svc, average='macro')
        mat_log= matthews_corrcoef(Y_train, y_pred_log)
        mat_svc= matthews_corrcoef(Y_train, y_pred_svc)
        
        f1_log=f1_score(Y_train, y_pred_log)
        f1_svc=f1_score(Y_train, y_pred_svc)

        #print('[0]logistic regression accuracy:',log)
        #print('[1]SVM accuracy:',svc)
        #print('[2]logistic regression recall:',rec_log)
        #print('[3]SVM regression recall:',rec_svm)

        #recall_score(y_true, y_pred, average='macro')
        return log,svc,rec_log,rec_svc,mat_log,mat_svc,f1_log,f1_svc

In [1]:
def models_crossval(Xtrain_cv,Ytrain_cv,ns):
    output_dict={}
    ac_log_list = []
    ac_svc_list = []
    rec_log_list = []
    rec_svc_list = []
    mat_log_list = []
    mat_svc_list = []
    f1_log_list = []
    f1_svc_list = []
    
    for i in range(ns):
        log,svc,r_log,r_svc,m_log,m_svc,f1_log,f1_svc = models(Xtrain_cv[i],Ytrain_cv[i])
        ac_log_list.append(log)
        ac_svc_list.append(svc)
        rec_log_list.append(r_log)
        rec_svc_list.append(r_svc)
        mat_log_list.append(m_log)
        mat_svc_list.append(m_svc)
        f1_log_list.append(m_log)
        f1_svc_list.append(m_svc)
        
    ac_log_array = np.array(ac_log_list)
    ac_svc_array = np.array(ac_svc_list)
    r_log_array = np.array(rec_log_list)
    r_svc_array = np.array(rec_svc_list)
    m_log_array = np.array(mat_log_list)
    m_svc_array = np.array(mat_svc_list)
    f1_log_array = np.array(f1_log_list)
    f1_svc_array = np.array(f1_svc_list)
    
    
    ac_log_mean = np.mean(ac_log_array)
    ac_svc_mean = np.mean(ac_svc_array)
    r_log_mean = np.mean(r_log_array)
    r_svc_mean = np.mean(r_svc_array)
    m_log_mean = np.mean(m_log_array)
    m_svc_mean = np.mean(m_svc_array) 
    f1_log_mean = np.mean(f1_log_array)
    f1_svc_mean = np.mean(f1_svc_array)  
        
    #print('average logistic regression accuracy:'+str(ac_log_mean))
    #print('average SVM accuracy:'+str(ac_svc_mean))
    #print('average logistic regression recall:'+str(r_log_mean))
    #print('average SVM recall:'+str(r_svc_mean))
    #print('average logistic regression matthews_corrcoef:'+str(m_log_mean))
    #print('average SVM matthews_corrcoef:'+str(m_svc_mean))
    #print('average logistic regression f1 score:'+str(f1_log_mean))
    #print('average SVM f1 score:'+str(f1_svc_mean))
    
    output_dict['average logistic regression accuracy:']=ac_log_mean
    output_dict['average SVM accuracy:']=ac_svc_mean
    output_dict['average logistic regression recall:']=r_log_mean
    output_dict['average SVM recall:']=r_svc_mean
    output_dict['average logistic regression matthews_corrcoef:']=m_log_mean
    output_dict['average SVM matthews_corrcoef:']=m_svc_mean
    output_dict['average logistic regression f1 score:']=f1_log_mean
    output_dict['average SVM f1 score:']=f1_svc_mean
    
    return output_dict
    #return ac_log_mean,ac_svc_mean,r_log_mean,r_svc_mean, m_log_mean, m_svc_mean, f1_log_mean, f1_svc_mean

In [31]:
def svm_function(X,y):
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 20)
    from sklearn.svm import SVC
    svc_model = SVC()
    svc_model.fit(X_train, y_train)
    y_predict = svc_model.predict(X_test)
    from sklearn.metrics import classification_report, confusion_matrix
    cm = np.array(confusion_matrix(y_test, y_predict, labels=[1,0]))
    confusion = pd.DataFrame(cm, index=['AMP', 'non_AMP'],
                         columns=['predicted_AMP','predicted_nonAMP'])
    class_report = classification_report(y_test,y_predict)
    print(class_report)
    return svc_model,confusion,class_report,y_predict

In [32]:
#for each bacterium we need X, y, and klist

In [41]:
#create dictionary for each bacterium
def bacterium_load(bacterium_file_name,data_dict,bacterium_name):
    ddf=pd.read_csv(bacterium_file_name)
    df2=ddf.drop(columns=['Unnamed: 0'])
    df_norm=normalized_df(df2)
    y = df2['MIC_0']
    dff2 = df2.drop(columns=['AMP_Name','MIC','MIC_0'])
    klist = np.array(dff2.columns)
    #first z score
    from scipy import stats

    X = stats.zscore(dff2, axis = 1, ddof = 1, nan_policy = 'raise')
    data_dict[bacterium_name] = list(range(3))
    # also need: featuredict for 
    data_dict[bacterium_name][0] = X
    data_dict[bacterium_name][1] = y
    data_dict[bacterium_name][2] = klist
    return data_dict

In [42]:
def redundant_filter(all_des):
    redundant_cols = [ ]

    for column1 in all_des:
        for column2 in all_des:
            if column1!=column2:
                Cor_sp=stats.spearmanr(all_des[column1], all_des[column2])
                if Cor_sp[0]>0.95:
                    #print(column1,column2, Cor_sp[0] , Cor_sp[1] )
                    if column1 not in redundant_cols:
                        redundant_cols.append(column1)
                        #print(redundant_cols)
    return redundant_cols

In [44]:
def hist_plotter(ddf2,feature_name,num):

    from statsmodels.stats.weightstats import ztest as ztest
    plt.style.use('classic')

    plt.figure()
    nonAMP_feature=ddf2[ddf2['MIC_0']==0][feature_name]
    AMP_feature=ddf2[ddf2['MIC_0']==1][feature_name]
    x1 = list(nonAMP_feature)

    x2 = list(AMP_feature)
    plt.hist(x2, density=True, bins=num, color='green', label='AMP')  # density=False would make counts
    plt.hist(x1, density=True, bins=num, color='red', alpha = 0.5, label='nonAMP')  # density=False would make counts
    
    plt.ylabel('Frequency', fontsize = 20)
    plt.xlabel(feature_name, fontsize = 20);
    plt.legend(loc='upper right', fontsize =15)
    plt.title((ztest(x1, x2, value=0) ))
    plt.savefig('hist_' + feature_name + '.pdf')

    

In [45]:
def length_filter(ddf,seq_len):

    ll_list= []
    for i in range(len(ddf['sequence'])):
    #print(i)
        ll=len(ddf['sequence'].iloc[i])
    #print(ll)
        ll_list.append(ll)

    df['seq_length']=ll_list
    
    return df[df['seq_length']>seq_len]
    

In [39]:
def feature_ztest(df2):
    import numpy as np
    import pingouin as pg
    feature_z = {}
    i=0
    for columns in df2:
        nonAMP_feature=df2[df2['MIC_0']==0][columns]
        AMP_feature=df2[df2['MIC_0']==1][columns]
        x1 = list(nonAMP_feature)
        x2 = list(AMP_feature)
        #z_val=ztest(x1, x2, value=0) 
        z_val=pg.mwu(x1, x2, alternative='two-sided')
        #if np.abs(z_val[0])>critical_z:
        #    i+=1
        print(columns, np.abs(z_val[0]),z_val[1],i)
        #    feature_z[columns] = np.abs(z_val[0])

    return 

In [40]:
def mann_whitney_test(df1):

#z_dict=feature_ztest(df1)
    import numpy as np
    import pingouin as pg
    feature_u = []
    
    for columns in df1.columns:
        nonAMP_feature=df1[df1['MIC_0']==0][columns]
        AMP_feature=df1[df1['MIC_0']==1][columns]
        x1 = list(nonAMP_feature)
        x2 = list(AMP_feature)
        #z_val=ztest(x1, x2, value=0) 
        z_val=pg.mwu(x1, x2, alternative='two-sided')
        #tmp1 = np.float(z_val['RBC'])
        #tmp2 = np.float(z_val['CLES'])
        #print(columns)
        #print(columns,z_val)
        #print(columns,np.float(z_val['p-val']),np.float(z_val['U-val']),np.float(z_val['RBC']),np.float(z_val['CLES']))

        if np.float(z_val['RBC'])<-0.3  and np.float(z_val['CLES'])>0.5:
            print(columns,np.float(z_val['p-val']),np.float(z_val['U-val']),np.float(z_val['RBC']),np.float(z_val['CLES']))
            feature_u.append(columns)
        
        
    return feature_u


In [ ]:
#df=pd.read_csv("AT_descriptors2.csv")
def distance_PP(network_name):

    df = pd.read_csv(network_name + "_descriptors.csv")
    df1 = df.drop(columns = ['Unnamed: 0','AMP_Names'])
    #df1.index = df['AMP_Name']

    scaler = MinMaxScaler()
    df_normalized = scaler.fit_transform(df1)
    scaled_features_df = pd.DataFrame(df_normalized, index=df1.index, columns=df1.columns)
    scaled_features_df['AMP_Name']=df['AMP_Names']
    scaled_features_df.set_index("AMP_Name", inplace = True)
    ll=df['AMP_Names'].tolist()
    
    with open(path_h+network_name+'_pair.txt') as file:
        lines = [line.rstrip() for line in file]
            
        P1_list=[]
        P2_list=[]
        PPI_list=[]
        PPI_dict={}
        for items in lines:
            x = items.split("\t")
            #print(x)
            if x[0] in ll:
                if x[1] in ll:
                    if x[2]=='+':
                        PPI_dict[x[0],x[1]]=1
                    else:
                        PPI_dict[x[0],x[1]]=0    
            
    
    # construct distance matrix for PPI
    df_distance= pd.DataFrame()
    df_distance['PPI_names']=list(PPI_dict.keys())
    df_distance['PPI']=list(PPI_dict.values())

    for columns in df1:
        scaled_features_df2=scaled_features_df[columns]
        xx=[]
        for x , y in PPI_dict.items():
            amp_distance=Euclidean_distance (scaled_features_df2, x[0], x[1])
            xx.append(amp_distance)
        df_distance[columns]=xx
    #print(df_distance)
    
    df_distance.to_csv(network_name+'_distance.csv')
    
    return df_distance.head()


In [1]:
def selected_features_plotter(featuredict,network_name,feature_selection_method,fonts,dr):

    fig_title = network_name + "_" + feature_selection_method + "_Feature_Importance.pdf"
    
    if feature_selection_method == 'lasso':
        bar_color = 'g'
        sub_title='(a)'
    elif feature_selection_method == 'svm':
        bar_color = 'r'
        sub_title='(b)'


    featuredict = sorted(featuredict.items(), key=lambda x:x[1])
    feature_name=[]
    feature_coef=[]

    for items in featuredict:
        #print(items[0], items[1])
        feature_name.append(items[0])
        feature_coef.append(items[1])
    xx=feature_name
    yy=feature_coef
    # Plot of feature importance
    import matplotlib.pyplot as plt
    import seaborn as sns
    fig, ax = plt.subplots()

    #plt.figure(figsize=(7, 7))
    #dr=0.2
    plt.barh(xx, yy, color=bar_color)
    #ax.set_xticks(np.arange(-max(yy),max(yy),0.1))
    #ax.set_xticks(np.arange(-np.round(max(yy),2),np.round(max(yy),2)+dr,dr))
    ax.set_xticks(np.arange(np.round(min(yy),2),np.round(max(yy),2)+dr,dr))

    plt.grid(False)

    plt.xlabel("Feature Importance",  fontsize=fonts)
    plt.figtext(0.45,0.94,sub_title,size=25)
    plt.tick_params(labelsize=8)
    plt.savefig(fig_title, bbox_inches = 'tight',
        pad_inches = 0)
    plt.show()

In [ ]:
def network_vis(network_name,file_path):
    import pandas as pd
    import networkx as nx
    import matplotlib.pyplot as plt

    with open(path_h + network_name+'_pair' +'.txt') as file:
        lines = [line.rstrip() for line in file]
    
    P1_list=[]
    P2_list=[]
    PPI_list=[]
    PPI_list_name=[]

    for items in lines:
        x = items.split("\t")
        P1_list.append(x[0])
        P2_list.append(x[1])
        PPI_list.append(x[2])
        PPI_list_name.append(x[0] + '-'+ x[1])

    
    df= pd.DataFrame()
    df['Protein1']=P1_list
    df['Protein2']=P2_list
    df['PIP']=PPI_list
    df['PIP_name']=PPI_list_name

    df4=df[df.PIP=='+']
    
    
    G4 = nx.from_pandas_edgelist(df4, source='Protein1', target='Protein2')
    fig, ax = plt.subplots(figsize=(6,6))
    pos = nx.random_layout(G4)
    nx.draw(G4,pos, ax=ax, with_labels=True,font_size=11,font_color  ='k')
    fig.savefig(network_name+'_network.pdf', bbox_inches='tight')
 
    return G4

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, matthews_corrcoef, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC



def models_overfitting(X_train, Y_train, X_test, Y_test):
    # Feature scaling
    #scaler = StandardScaler().fit(X_train)
    #X_train_scaled = scaler.transform(X_train)
    #X_test_scaled = scaler.transform(X_test)

    # Initialize and train Logistic Regression
    log1 = LogisticRegression(random_state=0).fit(X_train, Y_train)
    
    # Initialize and train SVM
    svc_model = SVC().fit(X_train, Y_train)
    
    # Evaluate models
    log_metrics = evaluate_model(log1, X_train, Y_train, X_test, Y_test)
    svc_metrics = evaluate_model(svc_model, X_train, Y_train, X_test, Y_test)

    # Print metrics
    for metric, value in log_metrics.items():
        print(f"Logistic Regression {metric}: {value:.4f}")
    print("\n")
    for metric, value in svc_metrics.items():
        print(f"SVC {metric}: {value:.4f}")
    print("\n")

    # Return as dictionaries for clarity
    return {'logistic_regression': log_metrics, 'svc': svc_metrics}

# Usage:
# results = models_overfitting(X_train, Y_train, X_test, Y_test)


In [ ]:
def load_dataset(network_name,path):
    if path is None:
        raise ValueError("The path is not specified.")
        
    df0 = pd.read_csv(path+network_name + "_distance.csv")
    df = df0.drop(columns = ['Unnamed: 0'])
    df2=df.drop(columns=['PPI_names'])
    df3=df.drop(columns=['PPI_names','PPI'])
    X = np.array(df3)
    y = np.array(df2['PPI'])
    klist = np.array(df3.columns)
    
    return np.array(X), np.array(y), klist


In [ ]:

def evaluate_model(model, X_train, Y_train, X_test, Y_test):
    """Evaluate a model's performance on training and test data."""
    # Training predictions
    y_train_pred = model.predict(X_train)
    # Test predictions
    y_test_pred = model.predict(X_test)

    # Metrics calculation
    metrics = {
        'train_accuracy': accuracy_score(Y_train, y_train_pred),
        'test_accuracy': accuracy_score(Y_test, y_test_pred),
        'train_recall': recall_score(Y_train, y_train_pred, average='macro'),
        'test_recall': recall_score(Y_test, y_test_pred, average='macro'),
        'train_mcc': matthews_corrcoef(Y_train, y_train_pred),
        'test_mcc': matthews_corrcoef(Y_test, y_test_pred),
        'train_f1': f1_score(Y_train, y_train_pred),
        'test_f1': f1_score(Y_test, y_test_pred)
    }
    
    return metrics


In [ ]:


def cross_species_evaluation(network1, network2, path):
    """
    Train models on data from one organism (network1) and evaluate on another (network2).
    
    Parameters:
    - network1: identifier or name for the first organism/network
    - network2: identifier or name for the second organism/network
    - path: directory path containing the datasets
    
    Returns:
    - A dictionary containing performance metrics for both logistic regression and SVC models
    """
    # Set up logging
    logging.basicConfig(level=logging.INFO)
    
    # Load datasets
    X_1, y_1, klist_1 = load_dataset(network1, path)
    X_2, y_2, klist_2 = load_dataset(network2, path)
    print(X_1.shape[1] , X_2.shape[1])

    # Check if the feature dimensions match
    if X_1.shape[1] != X_2.shape[1]:
        logging.error("Feature dimensions of the two datasets do not match!")
        return None
    
    # Cross-validation split (consider using K-fold in the future)
    ns = 1
    Xtrain_1, Xtest_1, Ytrain_1, Ytest_1 = cross_val_split(X_1, y_1, ns)
    # Train models
    #log,svc,rec_log,rec_svc,mat_log,mat_svc,f1_log,f1_svc = models(Xtrain_1[0], Ytrain_1[0])

    log_model = LogisticRegression(random_state=0).fit(Xtrain_1[0], Ytrain_1[0])
    
    svc_model = SVC().fit(Xtrain_1[0], Ytrain_1[0])
    
    
    
    # Evaluate models on network2 data
    svc_metrics = evaluate_model(svc_model, Xtrain_1[0], Ytrain_1[0], X_2, y_2)
    log_metrics = evaluate_model(log_model, Xtrain_1[0], Ytrain_1[0], X_2, y_2)

    return {
        'svc': svc_metrics,
        'log': log_metrics
    }

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

def train_evaluate_xgboost(X_train, Y_train, X_test, Y_test):

    # Initialize and train classifier
    # Initialize and train classifier with hyperparameters to prevent overfitting
    model = xgb.XGBClassifier(
        max_depth=5,            # Typical values range from 3 to 10
        min_child_weight=1,    # Increase to make the model more conservative
        subsample=0.8,         # Value less than 1.0
        colsample_bytree=0.9,  # Value less than 1.0
        gamma=0.01,            # Increase to make the model more conservative
        alpha=0.1,             # L1 regularization term (increase for more regularization)
        reg_lambda=1               # L2 regularization term (increase for more regularization)
    )       
    model.fit(X_train, Y_train)



    # Calculate accuracy
    #accuracy = accuracy_score(Y_test, Y_pred)
    xgb_metrics=evaluate_model(model, X_train, Y_train, X_test, Y_test)
    
    
    return xgb_metrics



def train_evaluate_xgboost2(X_train, Y_train, X_test, Y_test, colsample, gamma_value):
    # Initialize and train classifier with specified hyperparameters
    model = xgb.XGBClassifier(
        max_depth=5,            # Typical values range from 3 to 10
        min_child_weight=1,    # Increase to make the model more conservative
        subsample=0.8,         # Value less than 1.0
        colsample_bytree=colsample,  # Value less than 1.0
        gamma=gamma_value,            # Increase to make the model more conservative
        alpha=0.05,             # L1 regularization term (increase for more regularization)
        reg_lambda=0.01               # L2 regularization term (increase for more regularization)
    )   
    model.fit(X_train, Y_train)

    xgb_metrics=evaluate_model(model, X_train, Y_train, X_test, Y_test)

    
    return xgb_metrics





In [ ]:
#df=pd.read_csv("AT_descriptors2.csv")
def distance_PP2(network_name):

    df = pd.read_csv(network_name + "_descriptors.csv")
    df1 = df.drop(columns = ['Unnamed: 0','AMP_Name'])
    #df1.index = df['AMP_Name']

    scaler = MinMaxScaler()
    df_normalized = scaler.fit_transform(df1)
    scaled_features_df = pd.DataFrame(df_normalized, index=df1.index, columns=df1.columns)
    scaled_features_df['AMP_Name']=df['AMP_Name']
    scaled_features_df.set_index("AMP_Name", inplace = True)
    ll=df['AMP_Name'].tolist()
    
    with open(path_h+network_name+'_pair.txt') as file:
        lines = [line.rstrip() for line in file]
            
        P1_list=[]
        P2_list=[]
        PPI_list=[]
        PPI_dict={}
        for items in lines:
            x = items.split("\t")
            #print(x)
            if x[0] in ll:
                if x[1] in ll:
                    if x[2]=='+':
                        PPI_dict[x[0],x[1]]=1
                    else:
                        PPI_dict[x[0],x[1]]=0    
            
    
    # construct distance matrix for PPI
    df_distance= pd.DataFrame()
    df_distance['PPI_names']=list(PPI_dict.keys())
    df_distance['PPI']=list(PPI_dict.values())

    for columns in df1:
        scaled_features_df2=scaled_features_df[columns]
        xx=[]
        for x , y in PPI_dict.items():
            amp_distance=Euclidean_distance (scaled_features_df2, x[0], x[1])
            xx.append(amp_distance)
        df_distance[columns]=xx
    #print(df_distance)
    
    df_distance.to_csv(network_name+'_distance.csv')
    
    return df_distance.head()

In [ ]:
def load_network_data(network_name, path_h):
    
    df0 = pd.read_csv(path_h + network_name + "_distance.csv")
    df = df0.drop(columns=['Unnamed: 0'])
    df2 = df.drop(columns=['PPI_names'])
    df3 = df.drop(columns=['PPI_names', 'PPI'])
    
    XX = np.array(df3)
    yy = np.array(df2['PPI'])
    X=np.array(XX)
    y=np.array(yy)
    klist = np.array(df3.columns)
    
    return X, y,klist,df3